In [21]:
import pandas as pd
import numpy as np

raw_file = "/home/ec2-user/SageMaker/Team-5/Fiona/TN_allinone0112.csv"
df = pd.read_csv(raw_file)

print(df.shape)

## Converting heights
df['Heightin_to_cm'] = df['Heightin'] * 2.53999863
df['Height_cm'] = df['Heightin_to_cm'] + df['Heightcm']
df.insert(df.columns.get_loc('Height_cm') + 1, 'Height_cm_units', 'cm')
df.drop(['Heightin_to_cm', 'Heightcm', 'Heightin'], axis=1, inplace=True)

df['Weightlbs_to_kg'] = df['Weightlbs'] * 0.45359237
df['Weight_kg'] = df['Weightlbs_to_kg'] + df['Weightkg']
df.insert(df.columns.get_loc('Weight_kg') + 1, 'Weight_kg_units', 'kg')
df.drop(['Weightlbs_to_kg', 'Weightkg', 'Weightlbs'], axis=1, inplace=True)

## Converting glucose units and organizing columns
for column in df.columns:
    if 'mmol' in column.lower() or 'mgdl' in column.lower():
        new_column_name = column.replace('mmol', '_unit').replace('mgdl', '').replace('Mmoll', '_unit').replace('MgDL', '').replace('mmoll', '_unit')
        df.rename(columns={column: new_column_name}, inplace=True)
        
columns_to_process = ['AvgRecordedFastingGlucoses', 'AvgAllRecordedGlucoses', 'LowestGlucose', 'HighestGlucose']
df[columns_to_process] = df[columns_to_process].apply(lambda x: x.where(x >= 45, x * 18.018))

columns_to_convert = ['AvgRecordedFastingGlucoses_unit', 'AvgAllRecordedGlucoses_unit', 'LowestGlucose_unitl', 'HighestGlucose_unit']
df[columns_to_convert] = df[columns_to_convert].replace('mmol/L', 'mg/dl')

Hemoglobin_process = ['HemoglobinResults']
df[Hemoglobin_process] = df[Hemoglobin_process].apply(lambda x: x.where(x <= 30, x / 10))

Hematocrit_process = ['HematocritResults']
df[Hematocrit_process] = df[Hematocrit_process].apply(lambda x: x.where(x >= 1, x * 100))

# existing_column = 'HemoglobinResults'
# new_column_name = 'HemoglobinResults_Units'
# existing_column_index = df.columns.get_loc(existing_column)
# df.insert(existing_column_index + 1, new_column_name, "NaN")
# df.loc[df[existing_column].notna(), new_column_name] = "mg/dL"


column_to_move1 = 'AvgRecordedFastingGlucoses_unit'
target_position1 = 'AvgRecordedFastingGlucoses'
current_positions = df.columns.tolist()
current_positions.remove(column_to_move1)
target_index = current_positions.index(target_position1)
new_positions = current_positions[:target_index + 1] + [column_to_move1] + current_positions[target_index + 1:]
df = df[new_positions]

column_to_move2 = 'AvgAllRecordedGlucoses_unit'
target_position2 = 'AvgAllRecordedGlucoses'
current_positions = df.columns.tolist()
current_positions.remove(column_to_move2)
target_index = current_positions.index(target_position2)
new_positions = current_positions[:target_index + 1] + [column_to_move2] + current_positions[target_index + 1:]
df = df[new_positions]

df.replace('mg/dl', 'mg/dL', inplace=True, regex=True)

df.rename(columns={'LowestGlucose_unitl': 'LowestGlucose_unit'}, inplace=True)

print(df.shape)

/tmp/ipykernel_26601/3983451768.py:5: DtypeWarning: Columns (3,4,5,6,10,15,17,21,22,24,25,27,28,30,31,33,35,36,38,40,41,44,45,48,49,52,53,56,57,60,61,62,63,64,65,66,67,68,69,70,71,72,73,75,78,81,82,83,85,86,94,107,108,109,110,111,112,113,114,115,117,119,122,126,128,129,130,132,133,137,138,142,143,147,148,152,153,157,158,162,163,167,168,169,170,171,172,173,175,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,253,256,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,300,302,303,304,305,307,310,313,317,319,322,324,325,328,330,332,334,335,336,338,343,348,349,351,352,353,354,355,356,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,38

(259168, 661)
(259168, 661)


In [28]:
target_column = 'NeutrophilsResults'
unique_values = df[target_column].unique()
print(unique_values)

non_na_values = df[target_column].dropna().unique()
print("Non-NA values in", target_column, "column:", non_na_values)

target_column_index = df.columns.get_loc(target_column)

columns_on_left = df.columns[target_column_index - 1] if target_column_index > 0 else None
columns_on_right = df.columns[target_column_index + 1] if target_column_index < len(df.columns) - 1 else None

unique_values_left = df[columns_on_left].unique() if columns_on_left is not None else None
unique_values_right = df[columns_on_right].unique() if columns_on_right is not None else None

print("Unique values in columns on the left:", unique_values_left)
print("Unique values in columns on the right:", unique_values_right)

[      nan 4.890e+01 4.570e+01 4.790e+01 5.860e+01 5.920e+01 5.230e+01
 4.220e+01 5.650e+01 5.830e+01 5.660e+01 4.560e+01 5.320e+01 5.400e+01
 2.710e+01 4.070e+01 3.630e+01 3.000e+01 4.100e+01 3.570e+01 3.460e+01
 3.770e+01 5.450e+01 4.320e+01 5.160e+01 3.900e+01 6.240e+01 4.300e+01
 4.410e+01 6.160e+01 6.320e+01 5.940e+01 6.060e+01 5.100e+01 5.900e+01
 5.700e+01 4.400e+01 4.500e+01 5.300e+01 5.200e+01 4.900e+01 6.700e+01
 9.360e+01 4.000e+01 4.800e+01 4.550e+01 3.200e+00 5.240e+01 6.660e+01
 5.950e+01 5.250e+01 5.130e+01 5.120e+01 9.310e+01 6.710e+01 5.820e+01
 4.710e+01 6.300e+01 6.100e+01 6.380e+01 6.780e+01 5.360e+01 2.400e+00
 6.330e+01 5.990e+00 6.890e+01 4.260e+01 4.590e+01 4.370e+01 3.890e+01
 7.640e+01 7.110e+01 6.830e+01 9.400e+01 5.800e+01 6.000e+01 6.900e+01
 6.500e+01 6.600e+01 5.970e+01 5.500e+01 3.820e+01 7.350e+01 4.860e+01
 1.680e+00 7.960e+01 5.020e+01 4.870e+01 4.850e+01 3.780e+01 9.410e+01
 4.750e+01 4.580e+01 2.100e+00 5.190e+01 1.930e+00 9.330e+01 6.260e+01
 4.240